In [1]:
%run "/Users/shubhi.singal@insight.com/ADT"

In [2]:
counts = {}
maxid = (-1)
maxc = 0
for id in diagnoses['PATIENT_ID']:
    if id in counts:
        counts[id] = counts[id] + 1
        if maxc < counts[id]:
            maxc = counts[id]
            maxid = id
    else:
        counts[id] = 0
print(maxid)
print(maxc)

109
539

In [3]:
example_patient = diagnoses.loc[diagnoses.PATIENT_ID == 109]
#example_patient

In [4]:
example_in_order = example_patient.sort_values(by='RECORDED_DTM')
#example_in_order

In [5]:
word = ''
for code in example_in_order.CODE:
    if isinstance(code, str):
        word += code[0]
#word

In [6]:
ordered_diagnoses = diagnoses.sort_values(by='RECORDED_DTM')
words = {}
#Iterrows is pretty slow, or so I have been led to believe. May seek faster solution if this is too slow.
for record in ordered_diagnoses.itertuples():
    pid = record.PATIENT_ID
    code = record.CODE
    if not isinstance(code, str):
        continue
    if pid not in words:
        words[pid] = ''
    words[pid] = words[pid] + record.CODE[0]
    
pid_list = []
word_list = []
word_len_list = []
for pid in words:
    pid_list.append(pid)
    word_list.append(words[pid])
    word_len_list.append(len(words[pid]))
words_pdformat = {'PATIENT_ID': pid_list, 'WORD': word_list, 'WORD_LENGTH': word_len_list}
words_diagnoses = pd.DataFrame.from_dict(words_pdformat)
fig = words_diagnoses.WORD_LENGTH.hist(bins=16)
display(fig.figure)

In [7]:
#full_df = pd.read_csv("diagnoses_mlmd.csv")
diagnosis_counts = {}
for record in diagnoses.itertuples():
    pid = record.PATIENT_ID
    code = record.CODE
    if not isinstance(code, str):
        continue
    if pid not in diagnosis_counts:
        diagnosis_counts[pid] = 0
    diagnosis_counts[pid] = diagnosis_counts[pid] + 1


In [8]:
d_c= list(diagnosis_counts.values())

In [9]:
#fig = plt.figure(figsize=(12,5))
plt.hist(d_c, bins=200)
display(plt.show())
#display(fig1.figure)

In [10]:
counts = Counter(diagnosis_counts.values())
#for i in range(1,20):
    #print(counts[i])

In [11]:
#max(diagnosis_counts.values())
#Yields 509. Who the hell has 509 diagnoses?
max(diagnosis_counts.keys(), key=(lambda key: diagnosis_counts[key]))

Out[34]: 109

In [12]:
max_patient = diagnoses.loc[diagnoses.PATIENT_ID == 109].sort_values(by='RECORDED_DTM')
#max_patient

In [13]:
#Generate words for the whole dataset.
ordered_diagnoses = diagnoses.sort_values(by='RECORDED_DTM')
full_words = {}
#How will itertuples hold up now?
for record in ordered_diagnoses.itertuples():
    pid = record.PATIENT_ID
    code = record.CODE
    if not isinstance(code, str):
        continue
    if pid not in full_words:
        full_words[pid] = ''
    full_words[pid] = full_words[pid] + record.CODE[0]
    
full_pid_list = []
full_word_list = []
full_word_len_list = []
for pid in full_words:
    full_pid_list.append(pid)
    full_word_list.append(full_words[pid])
    full_word_len_list.append(len(full_words[pid]))
full_words_pdformat = {'PATIENT_ID': full_pid_list, 'WORD': full_word_list, 'WORD_LENGTH': full_word_len_list}
full_words_df = pd.DataFrame.from_dict(full_words_pdformat)


In [14]:
full_words_df.head()

,PATIENT_ID,WORD,WORD_LENGTH
0,82574,FCFEDIZRKJCILDZCE,17
1,21081,JJTFBJ,6
2,29156,JKJBJDJFBNE,11
3,12001,MNIFEIG,7
4,32096,VZFSSENSSSSF,12


In [15]:
#sklearn expects reals for all entries in an nparray, not strings.
#Solution: store each word in a separate data structure, then pass in rows containing the index of their word.
#Then take levenshtein distance of the strings

#can use the existing dataframe as the external ds
#want to make indices explicit in the rows
words_df_mlready = full_words_df.drop("WORD", axis=1)
words_df_mlready = words_df_mlready.assign(WORD_INDEX =list(range(words_df_mlready.shape[0])))

def lev_metric(x, y):
    #get the indices
    i, j = x[2], y[2]
    return levenshtein(full_words_df.WORD[i], full_words_df.WORD[j])

def lev_affinity(X):
  Pair_dist = pairwise_distances(X, metric=lev_metric, n_jobs=1)
  return Pair_dist

model = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='average', distance_threshold=0)
#distance threshold 0 for full dendrogram
#Average linkage chosen relatively arbintrarily from among single, complete, average

In [16]:
#code taken from sklearn website
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [17]:
#pairwise_distances(words_df_mlready) #, metric=lev_metric)#, n_jobs=-1)

In [18]:
#distance_matrix=np.array() 
#words_df_mlready.shape
#distance_matrix = lev_affinity(words_df_mlready)

In [19]:
dbscan(words_df_mlready, eps=5, min_samples=2, algorithm='auto')

Out[42]: (array([ 5960, 5962, 10787, 10791, 16153, 16155, 26026, 26029, 26684,
 26688, 28236, 28238, 35021, 35023, 36424, 36425]),
 array([-1, -1, -1, ..., -1, -1, -1]))